In [2]:
!pip install kagglehub

In [6]:
# Install required package if not already installed
# pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd

# Specify the file name exactly as it appears in the dataset
file_path = "StudentsPerformance.csv"  # Note the capital 'S' and 'P'

# Load the dataset using KaggleDatasetAdapter
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "spscientist/students-performance-in-exams",
    file_path
)

C:\Users\HP\AppData\Local\Temp\ipykernel_10496\596629967.py:12: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB
None


In [17]:
# Parental education level is linked with the highest average math score
avg_math_score = df.groupby("parental level of education")["math score"].mean()
print(avg_math_score)

parental level of education
associate's degree    67.882883
bachelor's degree     69.389831
high school           62.137755
master's degree       69.745763
some college          67.128319
some high school      63.497207
Name: math score, dtype: float64


In [21]:
# Score difference between males and females across all subjects
gender_math = df.groupby("gender")["math score"].mean()
gender_read = df.groupby("gender")["reading score"].mean()
gender_write = df.groupby("gender")["writing score"].mean()

print(f"Math Avg by Subject:\n{gender_math}\n")
print(f"Reading Avg by Subject:\n{gender_read}\n")
print(f"Writing Avg by Subject:\n{gender_write}\n")

Math Avg by Subject:
gender
female    63.633205
male      68.728216
Name: math score, dtype: float64

Reading Avg by Subject:
gender
female    72.608108
male      65.473029
Name: reading score, dtype: float64

Writing Avg by Subject:
gender
female    72.467181
male      63.311203
Name: writing score, dtype: float64



In [23]:
# Does completing the test preparation course improve performance in each subject?
preparation_math = df.groupby("test preparation course")["math score"].mean()
preparation_read = df.groupby("test preparation course")["reading score"].mean()
preparation_write = df.groupby("test preparation course")["writing score"].mean()

print(f"Maths:\n{preparation_math}\n")
print(f"Read:\n{preparation_read}\n")
print(f"Writing:\n{preparation_write}\n")

Maths:
test preparation course
completed    69.695531
none         64.077882
Name: math score, dtype: float64

Read:
test preparation course
completed    73.893855
none         66.534268
Name: reading score, dtype: float64

Writing:
test preparation course
completed    74.418994
none         64.504673
Name: writing score, dtype: float64



In [6]:
# Which combination of gender, lunch type, and test preparation status produces the top 10% of scores?

below90_math = df["math score"].quantile(0.90)
top10_math = df[df["math score"]>below90_math]

below90_read = df["reading score"].quantile(0.90)
top10_read = df[df["reading score"]>below90_read]

below90_write = df["writing score"].quantile(0.90)
top10_write = df[df["writing score"]>below90_write]

combination_math = top10_math.groupby(["gender","lunch", "test preparation course"])["math score"].agg(["count","mean"])
print(f"Maths: \n{combination_math}\n")

combination_read = top10_read.groupby(["gender","lunch", "test preparation course"])["reading score"].agg(["count","mean"])
print(f"Reading: \n{combination_read}\n")

combination_write = top10_write.groupby(["gender","lunch", "test preparation course"])["writing score"].agg(["count","mean"])
print(f"Writing: \n{combination_write}")

Maths: 
                                             count       mean
gender lunch        test preparation course                  
female free/reduced completed                    2  92.500000
       standard     completed                   18  91.833333
                    none                        15  93.000000
male   free/reduced completed                    4  91.250000
                    none                         5  89.600000
       standard     completed                   27  92.592593
                    none                        24  90.500000

Reading: 
                                             count       mean
gender lunch        test preparation course                  
female free/reduced completed                   12  92.666667
                    none                         4  90.250000
       standard     completed                   28  94.321429
                    none                        36  93.166667
male   free/reduced completed                    3 

In [42]:
# Does lunch type have a uniform impact across all race/ethnicity groups, or does its effect vary?
lunch_race = df.groupby("race/ethnicity")["lunch"].value_counts()
print(lunch_race)

race/ethnicity  lunch       
group A         standard         53
                free/reduced     36
group B         standard        121
                free/reduced     69
group C         standard        205
                free/reduced    114
group D         standard        167
                free/reduced     95
group E         standard         99
                free/reduced     41
Name: count, dtype: int64


In [5]:
# Correlation b/w reading/math and writing scores
read_corr_write = df["reading score"].corr(df["writing score"])
math_corr_write = df["math score"].corr(df["writing score"])

print(f"Reading score correlation score with writing score: {read_corr_write}")
print(f"Math score correlation score with writing score: {math_corr_write}")

Reading score correlation score with writing score: 0.9545980771462478
Math score correlation score with writing score: 0.8026420459498085


In [7]:
# Demographics of top 10% students

demograph_math = top10_math.groupby(["gender","race/ethnicity", "parental level of education"])["math score"].agg(["count","mean"])
print(f"Maths: \n{demograph_math}\n")

demograph_read = top10_read.groupby(["gender","race/ethnicity", "parental level of education"])["reading score"].agg(["count","mean"])
print(f"Reading: \n{demograph_read}\n")

demograph_write = top10_write.groupby(["gender","race/ethnicity", "parental level of education"])["writing score"].agg(["count","mean"])
print(f"Writing: \n{demograph_write}")

Maths: 
                                                   count        mean
gender race/ethnicity parental level of education                   
female group A        some high school                 1   92.000000
       group B        associate's degree               2   92.000000
                      bachelor's degree                1   97.000000
                      high school                      1   87.000000
                      master's degree                  1   90.000000
                      some college                     1   88.000000
       group C        associate's degree               3   92.666667
                      bachelor's degree                2   94.000000
                      some college                     3   87.666667
       group D        associate's degree               1   88.000000
                      bachelor's degree                2   91.000000
                      high school                      1   88.000000
                      mast

In [28]:
df["total_percentage"] = round((df["math score"] + df["reading score"] + df["writing score"])/3,2)

labels = ["Low", "Medium", "High"]
bins = [0, 60, 80, 100]

df["Performance Category"] = pd.cut(df["total_percentage"],bins=bins,labels=labels)
print(df["Performance Category"].value_counts())

Performance Category
Medium    513
Low       293
High      194
Name: count, dtype: int64


In [29]:
df.to_csv("students_performance.csv", index=False)